In [22]:
import os 
from dotenv import load_dotenv

# Loading Environment variables
load_dotenv()

True

In [23]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
groq_api_key = os.environ.get("GROQ_API_KEY")

In [24]:
from langchain_groq import ChatGroq

model = ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C5BCAA2920>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C5BCAE21D0>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [25]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Kunal Gaikwad MSc Data Science student.")])

AIMessage(content="Hello Kunal Gaikwad! It's nice to meet you. \n\nAs an MSc Data Science student, what are you most interested in exploring within the field?  \n\nI'm always eager to learn more about the exciting work being done in data science.\n", response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 29, 'total_tokens': 87, 'completion_time': 0.120366061, 'prompt_time': 0.002625255, 'queue_time': None, 'total_time': 0.12299131599999999}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d00e7c43-971b-41ba-a480-85e9b40081f3-0', usage_metadata={'input_tokens': 29, 'output_tokens': 58, 'total_tokens': 87})

In [26]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, My name is Kunal Gaikwad MSc Data Science student."),
        AIMessage(content="Hi Kunal Gaikwad!\n\nIt's great to meet you. Being an MSc Data Science student is awesome - it's such a fascinating and rapidly growing field. \n\nWhat are you most interested in exploring within data science?  Do you have any specific projects you're working on or areas you're particularly passionate about? I'd love to hear more about your journey! 😊  \n\n"),
        HumanMessage(content="Hey what is my name and what do I do??")
    ]
)

AIMessage(content="You are Kunal Gaikwad, and you are an MSc Data Science student!  \n\nIs there anything else you'd like to know or discuss?  Perhaps you have a data science question you'd like help with?  I'm here to assist in any way I can. 😊  \n", response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 134, 'total_tokens': 199, 'completion_time': 0.134156745, 'prompt_time': 0.009108603, 'queue_time': None, 'total_time': 0.143265348}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-03795c10-da9e-44b5-aa1b-5f51cbc729db-0', usage_metadata={'input_tokens': 134, 'output_tokens': 65, 'total_tokens': 199})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [27]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [28]:
config =  {
    "session_id": "chat1"
    }

In [31]:
response = with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Kunal Gaikwad MSc Data Science student.")
    ],
    config=config
)
response

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Kunal Gaikwad! \n\nIt's great to meet you.  An MSc in Data Science is an exciting path!  \n\nAre you working on any interesting projects right now, or is there anything specific about data science you'd like to discuss?  I'm happy to help in any way I can. 😄 \n\n", response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 119, 'total_tokens': 193, 'completion_time': 0.153426122, 'prompt_time': 0.008378375, 'queue_time': None, 'total_time': 0.161804497}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8e6cfe4d-1765-490b-a770-02d392ef3ae6-0', usage_metadata={'input_tokens': 119, 'output_tokens': 74, 'total_tokens': 193})

In [30]:
response.content

"Hello Kunal Gaikwad! \n\nIt's nice to meet you. That's great, you're studying MSc Data Science. It's a fascinating field. \n\nWhat can I help you with today? Are you working on any particular projects or have any questions about data science? 😊  \n\n"

In [32]:
config1 = { "configurable": {"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As an AI, I have no memory of past conversations and don't know your name. Could you please tell me? 😊\n"

In [33]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Nice to meet you, John!  How can I help you today? 👋  \n\n'

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is John, as you told me. 😊 \n\nIs there anything else I can help you with?\n'

In [35]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name")],
    config=config
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Kunal Gaikwad.  😊  \n\nI remember that from our earlier conversation!  \n\n\n\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [36]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [46]:
chain.invoke({"messages": [HumanMessage(content="Hi My name is Marry Poppins")]})

AIMessage(content="Hello, Marry Poppins! It's a pleasure to meet you.  What can I do for you today?  \n\nPerhaps you'd like to hear a story, solve a riddle, or just have a chat?  I'm ready for anything! 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 37, 'total_tokens': 97, 'completion_time': 0.124120717, 'prompt_time': 0.002992065, 'queue_time': None, 'total_time': 0.127112782}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8a4d5783-0141-470f-bcf7-feb8b3785117-0', usage_metadata={'input_tokens': 37, 'output_tokens': 60, 'total_tokens': 97})

In [47]:
config2 = {"configurable": {
    "session_id":
    "chat 3"
}}

In [50]:
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [51]:
response

AIMessage(content="I still don't know your name.  \n\nRemember, I'm like a fresh start every time we talk.  \n\nDo you want to tell me your name?  😊 \n", response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 141, 'total_tokens': 183, 'completion_time': 0.086251428, 'prompt_time': 0.006759158, 'queue_time': None, 'total_time': 0.093010586}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e72aa86d-7f22-43ea-aa19-02436a2e69ef-0', usage_metadata={'input_tokens': 141, 'output_tokens': 42, 'total_tokens': 183})

In [52]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [55]:
response = chain.invoke({
    "messages": [
        HumanMessage(content="Hare Krishna, my name is Vasudev")
    ],
    "language":"Marathi"
})
response.content

'Hare Krishna, Vasudev! मी तुमच्याशी बोलण्यास आनंदी आहे. तुम्हाला काय मदत करू शकतो? 🙏\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [ ]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [62]:
config3 = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I am Krish")],"language":"Hindi"},
    config=config
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते कृष! \n\nआप कैसे हैं?  \n'

In [60]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [63]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Marathi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते कृष! मी तुमच्याशी बोलण्यास आनंदित आहे! 😊  \n\nतुम्ही काय करत आहात?\n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [64]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

c:\Users\ashwi\kunal_programming\UdemySelfLearning\krish-genai-course\venv-genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ashwi\kunal_programming\UdemySelfLearning\krish-genai-course\venv-genai\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='I like vanilla ice cream'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [65]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages" : messages + [HumanMessage(content="What ice cream do I like")],
        "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences!  \n\nWhat's your favorite flavor? 😊🍦\n"

In [66]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2".  😊 \n\n\n\nIs there anything else I can help you with?\n'

In [67]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}

In [68]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"As a large language model, I have no memory of past conversations and I don't know your name.  What's your name? 😊  \n"

In [69]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'You asked "whats 2 + 2"  😊  \n\n\n\nIs there anything else I can help you with?\n'